We have ended this. Unfortunately we did not beat out 1.05 of cheating predictions... seems like a lot of data leakage in that one but could not beat it without it.

In [335]:
import pandas as pd
import numpy as np
item_categories = pd.read_csv("item_categories.csv")
items = pd.read_csv("items.csv")
sales_train = pd.read_csv("sales_train.csv")
shops = pd.read_csv("shops.csv")
test = pd.read_csv("test.csv")

In [336]:
item_categories.head()

,item_category_name,item_category_id
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [337]:
items.head()

,item_name,item_id,item_category_id
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [338]:
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [339]:
sales_train = sales_train[sales_train.date_block_num > 12]
sales_train.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
1366911,01.02.2014,13,27,15242,699.0,1.0
1366912,25.02.2014,13,27,15200,299.0,1.0
1366913,19.02.2014,13,27,15279,799.0,1.0
1366914,26.02.2014,13,27,15202,299.0,1.0
1366915,01.02.2014,13,27,14888,549.0,1.0


In [340]:
test.head()

,ID,shop_id,item_id
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


Understand how many different values we have in the data

In [341]:
print("Number of unique items:",items["item_name"].nunique())
print("Number of unique item categories:", item_categories["item_category_name"].nunique())
print("Number of unique shops:", shops["shop_name"].nunique())
print("Total # of days worth of data:", sales_train["date"].nunique())
print("Total # of obs:", sales_train.shape[0])

Number of unique items: 22170
Number of unique item categories: 84
Number of unique shops: 60
Total # of days worth of data: 638
Total # of obs: 1568938


Understand our missing data situation

In [342]:
print("Total missing in training data: ",sales_train.isnull().sum().sum())

Total missing in training data:  0


Add in text features

In [343]:
from sklearn import *
import nltk

#Text Features for item names
feature_cnt = 25
tfidf = feature_extraction.text.TfidfVectorizer(max_features=feature_cnt)
items['item_name_len'] = items['item_name'].map(len) #Lenth of Item Description
items['item_name_wc'] = items['item_name'].map(lambda x: len(str(x).split(' '))) #Item Description Word Count
txtFeatures = pd.DataFrame(tfidf.fit_transform(items['item_name']).toarray())
cols = txtFeatures.columns
for i in range(feature_cnt):
    items['item_name_tfidf_' + str(i)] = txtFeatures[cols[i]]
items.head()


,item_name,item_id,item_category_id,item_name_len,item_name_wc,item_name_tfidf_0,item_name_tfidf_1,item_name_tfidf_2,item_name_tfidf_3,item_name_tfidf_4,...,item_name_tfidf_15,item_name_tfidf_16,item_name_tfidf_17,item_name_tfidf_18,item_name_tfidf_19,item_name_tfidf_20,item_name_tfidf_21,item_name_tfidf_22,item_name_tfidf_23,item_name_tfidf_24
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40,41,14,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,!ABBYY FineReader 12 Professional Edition Full...,1,76,68,9,0.0,0.0,0.0,0.0,0.0,...,0.0,0.403761,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.483839
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40,45,26,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40,47,26,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,***КОРОБКА (СТЕКЛО) D,4,40,43,25,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [344]:
#text features for item categories
feature_cnt = 25
tfidf = feature_extraction.text.TfidfVectorizer(max_features = feature_cnt)
item_categories["item_category_name_len"] = item_categories["item_category_name"].map(len)
item_categories["item_category_name_wc"] = item_categories["item_category_name"].map(lambda x: len(str(x).split(" ")))
txtFeatures = pd.DataFrame(tfidf.fit_transform(item_categories["item_category_name"]).toarray())
cols = txtFeatures.columns
for i in range(feature_cnt):
    item_categories["item_category_name_tfidf_"  + str(i)] = txtFeatures[cols[i]]
item_categories.head()

,item_category_name,item_category_id,item_category_name_len,item_category_name_wc,item_category_name_tfidf_0,item_category_name_tfidf_1,item_category_name_tfidf_2,item_category_name_tfidf_3,item_category_name_tfidf_4,item_category_name_tfidf_5,...,item_category_name_tfidf_15,item_category_name_tfidf_16,item_category_name_tfidf_17,item_category_name_tfidf_18,item_category_name_tfidf_19,item_category_name_tfidf_20,item_category_name_tfidf_21,item_category_name_tfidf_22,item_category_name_tfidf_23,item_category_name_tfidf_24
0,PC - Гарнитуры/Наушники,0,23,3,0.0,0.0,0.0,1.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Аксессуары - PS2,1,16,3,0.0,0.0,0.0,0.0,0.780837,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Аксессуары - PS3,2,16,3,0.0,0.0,0.0,0.0,0.000000,0.780837,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Аксессуары - PS4,3,16,3,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Аксессуары - PSP,4,16,3,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [345]:
#text features for shop
feature_cnt = 25
tfidf = feature_extraction.text.TfidfVectorizer(max_features=feature_cnt)
shops['shop_name_len'] = shops['shop_name'].map(len)  #Lenth of Shop Name
shops['shop_name_wc'] = shops['shop_name'].map(lambda x: len(str(x).split(' '))) #Shop Name Word Count
txtFeatures = pd.DataFrame(tfidf.fit_transform(shops['shop_name']).toarray())
cols = txtFeatures.columns
for i in range(feature_cnt):
    shops['shop_name_tfidf_' + str(i)] = txtFeatures[cols[i]]
shops.head()

,shop_name,shop_id,shop_name_len,shop_name_wc,shop_name_tfidf_0,shop_name_tfidf_1,shop_name_tfidf_2,shop_name_tfidf_3,shop_name_tfidf_4,shop_name_tfidf_5,...,shop_name_tfidf_15,shop_name_tfidf_16,shop_name_tfidf_17,shop_name_tfidf_18,shop_name_tfidf_19,shop_name_tfidf_20,shop_name_tfidf_21,shop_name_tfidf_22,shop_name_tfidf_23,shop_name_tfidf_24
0,"!Якутск Орджоникидзе, 56 фран",0,29,4,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,1.000000
1,"!Якутск ТЦ ""Центральный"" фран",1,29,4,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.322815,0.0,0.689588,0.648274
2,"Адыгея ТЦ ""Мега""",2,16,3,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.498580,0.0,0.000000,0.000000
3,"Балашиха ТРК ""Октябрь-Киномир""",3,30,3,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.000000,0.0,0.000000,0.000000
4,"Волжский ТЦ ""Волга Молл""",4,24,4,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.423972,0.0,0.000000,0.000000


In [346]:
#Make Monthly
sales_train['date'] = pd.to_datetime(sales_train['date'], format='%d.%m.%Y')
sales_train['month'] = sales_train['date'].dt.month
sales_train['year'] = sales_train['date'].dt.year
train = sales_train.drop(['date','item_price'], axis=1)
train = train.groupby([c for c in train.columns if c not in ['item_cnt_day']], as_index=False)[['item_cnt_day']].sum()
train = train.rename(columns={'item_cnt_day':'item_cnt_month'})
#Monthly Mean
shop_item_monthly_mean = train[['shop_id','item_id','item_cnt_month']].groupby(['shop_id','item_id'], as_index=False)[['item_cnt_month']].mean()
shop_item_monthly_mean = shop_item_monthly_mean.rename(columns={'item_cnt_month':'item_cnt_month_mean'})
#Add Mean Feature
train = pd.merge(train, shop_item_monthly_mean, how='left', on=['shop_id','item_id'])
#Last Month (Oct 2015)
shop_item_prev_month = train[train['date_block_num']==33][['shop_id','item_id','item_cnt_month']]
shop_item_prev_month = shop_item_prev_month.rename(columns={'item_cnt_month':'item_cnt_prev_month'})
shop_item_prev_month.head()
#Add Previous Month Feature
train = pd.merge(train, shop_item_prev_month, how='left', on=['shop_id','item_id']).fillna(0.)
#Add lag variables... this is new
lag_data = train.pivot_table(index=["shop_id", "item_id"], columns = "date_block_num", values = "item_cnt_month").iloc[:,-6:].reset_index()
train = pd.merge(train, lag_data, how='left', on=['shop_id','item_id']).fillna(0.)
#Items features
train = pd.merge(train, items, how='left', on='item_id')
#Item Category features
train = pd.merge(train, item_categories, how='left', on='item_category_id')
#Shops features
train = pd.merge(train, shops, how='left', on='shop_id')
train.head()

,date_block_num,shop_id,item_id,month,year,item_cnt_month,item_cnt_month_mean,item_cnt_prev_month,28,29,...,shop_name_tfidf_15,shop_name_tfidf_16,shop_name_tfidf_17,shop_name_tfidf_18,shop_name_tfidf_19,shop_name_tfidf_20,shop_name_tfidf_21,shop_name_tfidf_22,shop_name_tfidf_23,shop_name_tfidf_24
0,13,2,472,2,2014,1.0,1.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.49858,0.0,0.0,0.0
1,13,2,482,2,2014,1.0,1.500000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.49858,0.0,0.0,0.0
2,13,2,485,2,2014,1.0,1.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.49858,0.0,0.0,0.0
3,13,2,786,2,2014,1.0,1.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.49858,0.0,0.0,0.0
4,13,2,791,2,2014,1.0,1.166667,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.49858,0.0,0.0,0.0


In [347]:
#remove excess variables... this is new
#train = train[train.date_block_num >= 32]
train = train.drop(['month','year', 'item_cnt_prev_month', 'item_cnt_month'], axis=1)
train["item_cnt_month"] = 0.

In [348]:
train.head()

,date_block_num,shop_id,item_id,item_cnt_month_mean,28,29,30,31,32,33,...,shop_name_tfidf_16,shop_name_tfidf_17,shop_name_tfidf_18,shop_name_tfidf_19,shop_name_tfidf_20,shop_name_tfidf_21,shop_name_tfidf_22,shop_name_tfidf_23,shop_name_tfidf_24,item_cnt_month
0,13,2,472,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.49858,0.0,0.0,0.0,0.0
1,13,2,482,1.500000,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.49858,0.0,0.0,0.0,0.0
2,13,2,485,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.49858,0.0,0.0,0.0,0.0
3,13,2,786,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.49858,0.0,0.0,0.0,0.0
4,13,2,791,1.166667,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.49858,0.0,0.0,0.0,0.0


In [349]:
#remove month year and date_block_num... this is new
#test['month'] = 11
#test['year'] = 2015
test['date_block_num'] = 34

#Add Mean Feature
test = pd.merge(test, shop_item_monthly_mean, how='left', on=['shop_id','item_id']).fillna(0.)
#Add Previous Month Feature
test = pd.merge(test, shop_item_prev_month, how='left', on=['shop_id','item_id']).fillna(0.)
#Add lag variables... this is new
test = pd.merge(test, lag_data, how='left', on=['shop_id','item_id']).fillna(0.)
#Items features
test = pd.merge(test, items, how='left', on='item_id')
#Item Category features
test = pd.merge(test, item_categories, how='left', on='item_category_id')
#Shops features
test = pd.merge(test, shops, how='left', on='shop_id')
test['item_cnt_month'] = 0.

#this is new
test = test.drop(["item_cnt_prev_month"], axis = 1)

test.head()

,ID,shop_id,item_id,date_block_num,item_cnt_month_mean,28,29,30,31,32,...,shop_name_tfidf_16,shop_name_tfidf_17,shop_name_tfidf_18,shop_name_tfidf_19,shop_name_tfidf_20,shop_name_tfidf_21,shop_name_tfidf_22,shop_name_tfidf_23,shop_name_tfidf_24,item_cnt_month
0,0,5,5037,34,1.444444,1.0,1.0,1.0,3.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,1,5,5320,34,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,2,5,5233,34,2.000000,3.0,2.0,0.0,1.0,3.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,3,5,5232,34,1.000000,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,4,5,5268,34,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [356]:
train.head()

,date_block_num,shop_id,item_id,item_cnt_month_mean,28,29,30,31,32,33,...,shop_name_tfidf_16,shop_name_tfidf_17,shop_name_tfidf_18,shop_name_tfidf_19,shop_name_tfidf_20,shop_name_tfidf_21,shop_name_tfidf_22,shop_name_tfidf_23,shop_name_tfidf_24,item_cnt_month
0,13,2,472,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.49858,0.0,0.0,0.0,0.0
1,13,2,482,1.500000,0.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.49858,0.0,0.0,0.0,0.0
2,13,2,485,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.49858,0.0,0.0,0.0,0.0
3,13,2,786,1.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.49858,0.0,0.0,0.0,0.0
4,13,2,791,1.166667,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.49858,0.0,0.0,0.0,0.0


In [351]:
[c for c in test.columns if c not in train.columns]

['ID']

In [352]:
for c in ['shop_name','item_name','item_category_name']:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(list(train[c].unique())+list(test[c].unique()))
    train[c] = lbl.fit_transform(train[c].astype(str))
    test[c] = lbl.fit_transform(test[c].astype(str))
    print(c)

shop_name
item_name
item_category_name


In [358]:
#this is new 
initial_train = train[train.date_block_num == 32]
col = [c for c in train.columns if c not in [32, 33]]
x1 = initial_train[col]
x1 = x1.rename(columns={31:"Lag_1", 30:"Lag_2", 29:"Lag_3", 28:"Lag_4"})
y1 = np.log1p(initial_train[32].clip(0.,20.))
int_train = train[train.date_block_num == 33]
col = [c for c in train.columns if c not in [33]]
x2 = int_train[col]
x2 = x2.drop([28], axis = 1).rename(columns={32:"Lag_1", 31:"Lag_2", 30:"Lag_3", 29:"Lag_4"})
y2 = np.log1p(int_train[33].clip(0.,20.))

reg = ensemble.ExtraTreesRegressor(n_estimators=25, n_jobs=-1, max_depth=15, random_state=18)
reg.fit(x1,y1)
print('RMSE:', np.sqrt(metrics.mean_squared_error(y2.clip(0.,20.),reg.predict(x2).clip(0.,20.))))
#full train
#reg.fit(train[col],train['item_cnt_month'].clip(0.,20.))

#this is new
col = [c for c in train.columns if c not in [28, 33]]
reg.fit(int_train[col],int_train[33].clip(0.,20.))
col = [c for c in train.columns]
testData = test[col]
testData = testData.drop([28,29], axis = 1).rename(columns={33:"Lag_1", 32:"Lag_2", 31:"Lag_3", 30:"Lag_4"})
test['item_cnt_month'] = reg.predict(testData).clip(0.,20.)
test[['ID','item_cnt_month']].to_csv('sales_submission.csv', index=False)

RMSE: 0.343333995305


In [98]:
from catboost import CatBoostRegressor
from multiprocessing import *

In [213]:

cb_model = CatBoostRegressor(iterations=100, learning_rate=0.2, depth=7, loss_function='RMSE', eval_metric='RMSE', random_seed=18, od_type='Iter', od_wait=20) 
cb_model.fit(x1, y1, eval_set=(x2, y2), use_best_model=True, verbose=False)
print('RMSE:', np.sqrt(metrics.mean_squared_error(y2.clip(0.,20.), cb_model.predict(x2).clip(0.,20.))))

#this is new
test = test.drop([28,29], axis = 1).rename(columns={33:"Lag_1", 32:"Lag_2", 31:"Lag_3", 30:"Lag_4"})
test['item_cnt_month'] += cb_model.predict(test)

#test['item_cnt_month'] += cb_model.predict(test[col])
test['item_cnt_month'] /= 2
#test[['ID','item_cnt_month']].to_csv('catboost_submission.csv', index=False)
#gives 1.06 which is still much better!

RMSE: 0.26702449088


In [215]:
test.head()

,ID,shop_id,item_id,date_block_num,item_cnt_month_mean,Lag_4,Lag_3,Lag_2,Lag_1,item_name,...,shop_name_tfidf_16,shop_name_tfidf_17,shop_name_tfidf_18,shop_name_tfidf_19,shop_name_tfidf_20,shop_name_tfidf_21,shop_name_tfidf_22,shop_name_tfidf_23,shop_name_tfidf_24,item_cnt_month
0,0,5,5037,34,1.444444,1.0,3.0,1.0,0.0,1195,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.834907
1,1,5,5320,34,0.000000,0.0,0.0,0.0,0.0,1254,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.405607
2,2,5,5233,34,2.000000,0.0,1.0,3.0,1.0,1239,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.887106
3,3,5,5232,34,1.000000,0.0,1.0,0.0,0.0,1238,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.032881
4,4,5,5268,34,0.000000,0.0,0.0,0.0,0.0,1245,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.218212


In [295]:
testData.head(11)

,date_block_num,shop_id,item_id,item_cnt_month_mean,Lag_4,Lag_3,Lag_2,Lag_1,item_name,item_category_id,...,shop_name_tfidf_16,shop_name_tfidf_17,shop_name_tfidf_18,shop_name_tfidf_19,shop_name_tfidf_20,shop_name_tfidf_21,shop_name_tfidf_22,shop_name_tfidf_23,shop_name_tfidf_24,item_cnt_month
0,34,5,5037,1.444444,1.0,3.0,1.0,0.0,1195,19,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,34,5,5320,0.000000,0.0,0.0,0.0,0.0,1254,55,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,34,5,5233,2.000000,0.0,1.0,3.0,1.0,1239,19,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,34,5,5232,1.000000,0.0,1.0,0.0,0.0,1238,23,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
4,34,5,5268,0.000000,0.0,0.0,0.0,0.0,1245,20,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
5,34,5,5039,1.833333,0.0,0.0,1.0,1.0,1197,23,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
6,34,5,5041,2.500000,0.0,0.0,3.0,2.0,1198,20,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
7,34,5,5046,2.000000,0.0,0.0,0.0,0.0,1202,55,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
8,34,5,5319,5.727273,3.0,2.0,3.0,0.0,1253,55,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
9,34,5,5003,0.000000,0.0,0.0,0.0,0.0,1184,20,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [359]:
test[["ID", "item_cnt_month"]].head(11)

,ID,item_cnt_month
0,0,1.261284
1,1,0.171719
2,2,1.588619
3,3,1.036878
4,4,0.381188
5,5,1.395303
6,6,1.713505
7,7,1.731033
8,8,2.267098
9,9,0.308053


In [ ]:
0,0.7401197913913928
1,0.05968561041541443
2,1.0522809184351274
3,0.21039392659773815
4,0.05968561041541443
5,0.3840327582515848
6,1.011074499336693
7,0.1268816300620252
8,1.7146556847921897
9,0.05968561041541443
10,2.5119794818279364
11,0.32090253123589824